In [59]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [60]:
# Load Dataset
data = pd.read_csv('odi_Matches_Data.csv')  # Update with correct path

In [61]:
# User input for venue type
venue_type = input("Would you like to base predictions on 'city', 'stadium', or 'country'? ").strip().lower()

venue_mapping = {
    "stadium": "Match Venue (Stadium)",
    "city": "Match Venue (City)",
    "country": "Match Venue (Country)"
}

venue_column = venue_mapping.get(venue_type, "Match Venue (Country)")

if venue_column not in data.columns:
    raise KeyError(f"'{venue_column}' column not found in dataset. Available columns: {data.columns}")

In [62]:
print(data[['match_winner', 'team1', 'team2']].head(20))  # BEFORE encoding

   match_winner        team1        team2
0     Australia    Australia  New Zealand
1     Sri Lanka  New Zealand    Sri Lanka
2     Australia  New Zealand    Australia
3     Australia    Sri Lanka    Australia
4   New Zealand      England  New Zealand
5   West Indies        India  West Indies
6     Sri Lanka    Sri Lanka        India
7         India  New Zealand        India
8   West Indies  West Indies    Australia
9   West Indies  West Indies     Pakistan
10    Australia      England    Australia
11      England    Australia      England
12    Australia      England    Australia
13      England    Australia      England
14  New Zealand  New Zealand     Pakistan
15      England  New Zealand      England
16          NaN      England  New Zealand
17      England  West Indies      England
18  West Indies      England  West Indies
19    Australia  New Zealand    Australia


In [63]:
# Shuffle team1 and team2 randomly
random_mask = np.random.rand(len(data)) > 0.5
data.loc[random_mask, ['team1', 'team2']] = data.loc[random_mask, ['team2', 'team1']].values

In [64]:
# Update match_winner after swapping
def update_winner(row):
    if row['match_winner'] == row['team1']:
        return row['team1']
    elif row['match_winner'] == row['team2']:
        return row['team2']
    return row['match_winner']

data.loc[random_mask, 'match_winner'] = data[random_mask].apply(update_winner, axis=1)

In [65]:
# Ensure all relevant columns are strings before encoding
data['team1'] = data['team1'].astype(str)
data['team2'] = data['team2'].astype(str)
data['toss_winner'] = data['toss_winner'].astype(str)
data['match_winner'] = data['match_winner'].astype(str)
data[venue_column] = data[venue_column].astype(str)  # Ensure venue column is string

# Initialize LabelEncoders
le_team = LabelEncoder()
le_venue = LabelEncoder()

# ✅ Fit LabelEncoder on ALL unique team names (avoids unseen label errors)
all_teams = pd.concat([data['team1'], data['team2'], data['toss_winner'], data['match_winner']]).unique()
le_team.fit(all_teams)  # Now contains all possible teams

# ✅ Encode teams correctly
data['team1'] = le_team.transform(data['team1'])
data['team2'] = le_team.transform(data['team2'])
data['toss_winner'] = le_team.transform(data['toss_winner'])
data['match_winner'] = le_team.transform(data['match_winner'])

# ✅ Encode venue correctly
data['venue'] = le_venue.fit_transform(data[venue_column])

# ✅ Feature Engineering
data['home_advantage'] = np.where(data['venue'] == data['team1'], 1, 0)
data['team1_strength'] = data.groupby('team1')['match_winner'].transform('mean')
data['team2_strength'] = data.groupby('team2')['match_winner'].transform('mean')
data['recent_form1'] = data.groupby('team1')['match_winner'].transform(lambda x: x.rolling(5, min_periods=1).mean())
data['recent_form2'] = data.groupby('team2')['match_winner'].transform(lambda x: x.rolling(5, min_periods=1).mean())

# ✅ Ensure match_winner is binary (1 if team1 wins, else 0)
data['match_winner'] = (data['match_winner'] == data['team1']).astype(int)

print("✅ Encoding Completed Successfully!")

✅ Encoding Completed Successfully!


In [66]:
print(data['match_winner'].value_counts())

match_winner
0    2422
1    2323
Name: count, dtype: int64


In [67]:
# Selecting Features and Target
features = ['home_advantage', 'toss_winner', 'team1_strength', 'team2_strength', 'recent_form1', 'recent_form2']
target = 'match_winner'

X = data[features]
y = data[target]

In [68]:
# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [69]:
class_ratio = data['match_winner'].value_counts()[0] / data['match_winner'].value_counts()[1]
model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, scale_pos_weight=class_ratio)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [70]:
# Function to Predict Match Outcome
def predict_match(team1_name, team2_name, venue_name, toss_winner):
    if team1_name not in le_team.classes_ or team2_name not in le_team.classes_:
        raise ValueError("Invalid team name provided.")
    if venue_name not in le_venue.classes_:
        raise ValueError("Invalid venue provided.")
    
    team1 = le_team.transform([team1_name])[0]
    team2 = le_team.transform([team2_name])[0]
    venue = le_venue.transform([venue_name])[0]
    home_advantage = 1 if venue == team1 else 0
    toss_winner = 1 if toss_winner == team1 else 0
    team1_strength = data[data['team1'] == team1]['team1_strength'].mean()
    team2_strength = data[data['team2'] == team2]['team2_strength'].mean()
    recent_form1 = data[data['team1'] == team1]['recent_form1'].mean()
    recent_form2 = data[data['team2'] == team2]['recent_form2'].mean()
    
    input_data = np.array([[team1_strength==0.8, team2_strength==0.75, home_advantage==1, toss_winner==1, recent_form1==0.9, recent_form2==0.8]])
    probability = model.predict_proba(input_data)[0][1]  # Probability of team1 winning
    print(f'Probability of {team1_name} Winning: {probability * 100:.2f}%')
    return probability

In [71]:
predict_match("India", "Australia", "Mumbai", "Australia")

Probability of India Winning: 63.94%


0.63944656